<a href="https://colab.research.google.com/github/Kimutua/LLM_Observability/blob/main/Ambers_AI_Agents_for_Aliens(AAAA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
 This notebook is designed to combine the use LangChain LLM Agents, chains and tools to provide the more relevant and detailed information on a topic of your choice.

## Goal
 For this notebook, let's say your research goal is to prove the existance of Aliens. In this notebook we will try the following:


*   Create a single shot
*   List item



In [8]:
#Install and Import Required Modules,  LLMs and Agent Tools
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

API Keys

In [9]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')  # https://platform.openai.com
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')  # https://serpapi.com/

#SerpAPI is an API call to scrape Google search result, it is the JSON representative of Google search result.

## Creating and Initializing the Agent

In [10]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [11]:
llm = OpenAI(temperature=0) # First, let's load the language model we're going to use to control the agent. Temp = 0 generally means less hallucinations and a more deterministic model

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


### Tools

In [12]:
#BashProcess has been moved to langchain experimental.To use this tool,
#install langchain-experimental with `pip install langchain-experimental`

!pip -q install langchain-experimental

In [13]:
tools = load_tools(["serpapi", "llm-math","wikipedia","terminal"], llm=llm, allow_dangerous_tools="true")
#The tools the agent will have access to are: [Google search, math, wikipedia, terminal]

Tools are functions that an agent calls. There are two important considerations here:



1.   Giving the agent access to the right tools
2.   Describing the tools in a way that is most helpful to the agent

Without both, the agent you are trying to build will not work. If you don't give the agent access to a correct set of tools, it will never be able to accomplish the objective. If you don't describe the tools properly, the agent won't know how to properly use them.

In [14]:
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [15]:
tools[1].name, tools[1].description #If you ever need to look up a tool

('Calculator', 'Useful for when you need to answer questions about math.')

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

LangChain provides the Chain interface for such "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

Chains allow us to combine multiple components together to create a single, coherent application. For example, we can create a chain that takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM. We can build more complex chains by combining multiple chains together, or by combining chains with other components.



In [16]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator: Useful for when you need to answer questions about math.\nwikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nterminal: Run shell commands on this Linux machine.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator, wikipedia, terminal]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input q

The core idea of agents is to use an LLM to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.


In [18]:
#Let's test our Agent with a few straight forward questions.
agent.run("What do we currently know about Felix Kimutua?")



> Entering new AgentExecutor chain...
 I should use wikipedia to search for information about Felix Kimutua
Action: wikipedia
Action Input: Felix Kimutua
Observation: No good Wikipedia Search Result was found
Thought: I should use search to look for information about Felix Kimutua
Action: Search
Action Input: Felix Kimutua
Observation: ['“Felix is a highly competent individual, skilled in data science and s/w engineering. He delivers excellent work and constantly looks for ways to learn new ...', "When it's a norm to make the world a better place, a software piece a time.", 'As a Product Manager, one crucial task is to lead in building solutions that resolve customer pain points.', 'Read writing from Felix Kimutua on Medium. Software Developer. Every day, Felix Kimutua and thousands of other voices read, write, and share important ...', 'Connect with Felix Kimutua. Felix Kimutua · 3 Followers. Software Developer. Follow. Following. Jacqueline Kingori · Richard Kyaka.', 'Conversation.

'Felix Kimutua is a software developer, product manager, and writer. He is skilled in data science and software engineering and is passionate about making the world a better place. He has 2 books on Goodreads and is currently working at Safaricom as a Developer - Enterprise Integration & Service Development.'

In [19]:
#Let's test our Agent with a few straight forward questions.
agent.run("What do we currently know about aliens?")



> Entering new AgentExecutor chain...
 I should search for recent news articles or scientific studies.
Action: Search
Action Input: "aliens"
Observation: ["After floating in space for 57 years, Lt. Ripley's (Sigourney Weaver) shuttle is found by a deep space salvage team. Upon arriving at LV-426, the marines find only one survivor, a nine year old girl named Newt (Carrie Henn). But even these battle-hardened marines with all the latest weaponry are no match for the hundreds of aliens that have invaded the colony.… MORE", 'Aliens type: R 1986 ‧ Horror/Action ‧ 2h 17m.', 'Aliens entity_type: film, tvm.', 'Aliens main_tab_text: Overview.', 'Aliens kgmid: /m/01f8hf.', 'Aliens release_date: July 14, 1986 (USA).', 'Aliens director: James Cameron.', 'Aliens prequel: Alien.', 'Aliens distributed_by: 20th Century Studios.', "Aliens based_on: Characters; by - Dan O'Bannon; Ronald Shusett.", "... aliens rising up to the surface, are cut out. The scene in the movie's ending, where the alien quee

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Aliens (film)
Summary: Aliens is a 1986 science fiction action film written and directed by James Cameron. It is the sequel to the 1979 science fiction horror film Alien, and the second film in the Alien franchise. Set in the far future, it stars Sigourney Weaver as Ellen Ripley, the sole survivor of an alien attack on her ship. When communications are lost with a human colony on the moon where her crew first saw the alien creatures, Ripley agrees to return to the site with a unit of Colonial Marines to investigate. Michael Biehn, Paul Reiser, Lance Henriksen, and Carrie Henn are featured in supporting roles.
Despite the success of Alien, its sequel took years to develop due to lawsuits, a lack of enthusiasm from 20th Century Fox, and repeated management changes. Although relatively inexperienced, Cameron was hired to write a story for Aliens in 1983 on the strength of his scripts for The Terminator (1984) and Rambo: First Blood Part II (1985). The project stalled a

'The Alien franchise is a science fiction horror and action media franchise centered on the original film series, which began with Alien in 1979 and has since expanded to include sequels, prequels, and spin-offs. The franchise has been praised for its action and storytelling, and has become a cultural phenomenon.'

In [20]:
agent_output_UFO_expert = agent.run("Who is the leading expert on UFOs?")



> Entering new AgentExecutor chain...
 I should search for this information
Action: Search
Action Input: Leading expert on UFOs
Observation: ['UFO expert: Removing stigma around UFOs will lead to more answers. 117K views · 8 months ago #CNN #News ...more ...', '... Impact x Nightline" looks at the public\'s long obsession over UFOs and aliens ... Public\'s UFO obsession has experts, others ... MORE: ...', '... UFOs are linked to extraterrestrial aliens) ... 1949), member of CSICOP\'s UFO subcommittee, a leading UFO skeptic/debunker. ... "UFO expert has little patience for ...', 'UAP are better known to the public as unidentified flying objects, or UFOs. Nelson during a news conference also gave his personal opinion ...', "Leading Expert On UFOs and Aliens Interview - Philip Mantle Join TruthSeekah's Online School of the Mystics! ➡️ https://www.Patreon.com/join/TruthSeekah ...", 'Within the UFO community he is often regarded as one of the preeminent experts ... UFOs and the ... leadin

In [21]:
agent.run("Take the year" + agent_output_UFO_expert + "died and add 100 years. Will this be the year we prove aliens exist?")



> Entering new AgentExecutor chain...
 This question is asking for a prediction based on current information, so I should use a search engine to gather more information.
Action: Search
Action Input: "UFO experts"
Observation: ['MORE: Pentagon\'s \'UFO\' tracking efforts still find no alien origins ... "They\'re learning advanced interviewing techniques. They\'re learning ...', 'Fowler (b. 1934), long-time UFO investigator, details one of the best multiple witness alien abduction cases on record, author of The Andreasson Affair and The ...', "Harvard expert debunks alien, UFO sightings, says 'they wouldn't survive' · 'It would take about a billion years to cross from one side of the ...", 'NASA names chief of UFO research; panel sees no alien evidence. By Joey Roulette. September 14, 202312:41 PM PDTUpdated 6 months ago.', "Independent group of experts and scientists releases 33-page report on UFOs ... ufo navy vpx. US Navy. Pentagon launches 'one-stop shop' website ...", 'UFO journal

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4098 tokens (3842 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Remember Tools are functions that agents can use to interact with the world. These tools can be generic utilities (e.g. search), other chains, or even other agents.

Note how our single Agent is deciding between using Search, Math or Wikipedia to deliver the best response.




In [22]:
agent.run("Is" + agent_output_UFO_expert + "dead?")



> Entering new AgentExecutor chain...
 I should search for information on the individuals mentioned and their expertise in the field of UFOs.
Action: Search
Action Input: "David Grusch UFO expert", "Ryan Graves UFO expert", "David Fravor UFO expert", "Luis Elizondo UFO expert"
Observation: ['The U.S. is concealing a longstanding program that retrieves and reverse engineers unidentified flying objects, a former Air Force ...', "The Pentagon's report on UFOs, which said there's no evidence of aliens or secret spacecraft retrieval programs, was ripped by ...", 'UFO research, psychic and inter-dimensional operations were crammed into a report slammed as "threadbare lies"', 'The Pentagon UFO videos are selected visual recordings of FLIR targeting from United States Navy fighter jets based aboard aircraft carriers USS Nimitz and ...', '(Runtime 1:02). Read. It was late at night and the sky was full of stars. Jensen Lovelett often spent nights gazing at the constellations ...', "A top Penta

'No, there is no one definitive leading expert on UFOs and the study of UFOs is a highly debated and controversial topic.'

## Understanding the outcomes of the Model
 The use of a single Agent with tools is useful, but is starting to hallucinate. If we want correct and reliable information, maybe we should try to look at alternative words for UFOs and aliens...

In [23]:
#ask questions that can help you search for the most relevant research
agent.run("What are alternative names for UFOs?")



> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: "alternative names for UFOs"
Observation: ['Find 5 different ways to say UFO, along with antonyms, related words, and example sentences at Thesaurus.com.', 'Unidentified aerial phenomena offer a lesson on the residue problem in science.', 'A former intelligence officer-turned-whistleblower and two pilots testified before Congress about the strange, seemingly inexplicable ...', 'An unidentified flying object (UFO), or unidentified anomalous phenomenon (UAP), is any perceived aerial phenomenon that cannot be immediately identified or ...', 'UFOs: Unidentified flying objects, often assumed to be alien spacecraft · UAPs: Unidentified aerial phenomena, used to describe unexplainable ...', 'Review the search terms in response to requests for information on Unidentified Flying Objects (UFOs) and Paranormal Events, where no responsive material has ...', "Look, up in the sky! I

'The alternative names for UFOs are unidentified aerial phenomena (UAP), unidentified flying objects (UFOs), and unidentifiable things in the sky.'

Asking multiple questions at once.

In [26]:
#ask multiple questions at once
agent.run("How are Foo Fighters a name for a UFO? And is Dave Grohl human?")



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-instruct in organization org-zsHnqW9yBXgEzF2tYfhwqBTm on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

Wow, I learned something new, I really didn't think Dave Grohl was human... It is good to know a UFO is now being referred to as a UAP. I will keep that in mind for future research.

Now let's look up some outside resources to get answers about UFOs / UAPs...

In [ ]:
agent.run("Are there annual UAP reports?")


In [ ]:
agent.run("Where can I download the '2022 ANNUAL REPORT ON UNIDENTIFIED AERIAL PHENOMENA'? What is the url?")


In [ ]:
agent.run("Is there a pdf file for that report?")

Now that I know there is a full report available, I'm going to download it.

However, before I read through the entire report, I'm going to summarize it and then train a chat bot on it to make sure it contains the information I need.

In [ ]:
pip install "unstructured[all-docs]" #let's save the pdf via URL


In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

In [ ]:
urls = ["https://www.dni.gov/files/ODNI/documents/assessments/Unclassified-2022-Annual-Report-UAP.pdf"] #location of the report

In [ ]:
loader = UnstructuredURLLoader(urls=urls)

TOP_SECRET_FILE = loader.load() #clearly I need to save this as a top secret file

Time to summarize our top secret report.

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.question_answering import load_qa_chain

def doc_summary(docs):
    print (f'You have {len(docs)} document(s)')

    num_words = sum([len(doc.page_content.split(' ')) for doc in docs])

    print (f'You have roughly {num_words} words in your docs')
    print ()
    print (f'Preview: \n{docs[0].page_content.split(". ")[0]}')

In [ ]:
doc_summary(TOP_SECRET_FILE)

In [ ]:
chain = load_summarize_chain(llm, chain_type="map_reduce", verbose=True)
chain.run(TOP_SECRET_FILE) #This will only work if we do not exceed the token limit

Oh no, our document is too long if we load it all at once! So we will need to chunk the file with LangChains text splitter.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 600,
    chunk_overlap = 0
)
TOP_SECRET_FILE = text_splitter.split_documents(TOP_SECRET_FILE)
doc_summary(TOP_SECRET_FILE)

In [ ]:
chain.run(TOP_SECRET_FILE[1:40])


Great! This report seems to be what I'm looking for on regards to real information on UFOs. I don't really want to read the entire report to see if it can answer questions for an article I'm writing. Mainly I want to see if this reports tells us if we are in danger (from what we currently know about UFOs) and if the report provides evidence that aliens exist. Before I read the full report, let's just ask the report if it can answer my questions.

## Q&A: Map Re-Rank
The map re-rank documents chain runs an initial prompt on each document, that not only tries to complete a task but also gives a score for how certain it is in its answer. The highest scoring response is returned.

In [ ]:
chain = load_qa_chain(llm, chain_type="map_rerank", verbose=True, return_intermediate_steps=True)
#ask the report your important questions
query = "Are UAPs a danger to our safety? Does this mean that aliens are real?"

result = chain({"input_documents": TOP_SECRET_FILE[1:40], "question": query}, return_only_outputs=True)

In [ ]:
result['output_text']

Darn! But 1 out of 2 answers isn't too bad.

Let's check intermediate steps first just to check that these answers are reliable, as well as where exactly in the document chucks the answers were found.

In [ ]:
result['intermediate_steps']

Since reports and websearching hasn't gotten us all the answers we need about aliens, let's see if eye-witness testimonies from YouTube will help!

## We are now going to pull videos from Youtube and summarize their findings. We can achieve this with the OpenAIWhisperParser, which will use the OpenAI Whisper API to transcribe audio to text, and the OpenAIWhisperParserLocal for local support and running on private clouds or on premise.

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [ ]:
from langchain.document_loaders.parsers import OpenAIWhisperParser


We will use yt_dlp to download audio for YouTube urls.

We will use pydub to split downloaded audio files (such that we adhere to Whisper API's 25MB file size limit).

In [ ]:
pip install yt_dlp


In [ ]:
pip install pydub


In [ ]:
pip install librosa

# YouTube url to text


In [ ]:
pip install youtube_search

In [ ]:
from langchain.tools import YouTubeSearchTool

In [ ]:
Agent_youtube = YouTubeSearchTool()

In [ ]:
Agent_youtube.run("UFO breaking news,3") #search YouTube for the top 3 breaking news videos on UFOs

In order to visit Youtube and check out these videos you can enter the URLs directly into the search window by adding https://www.youtube.com before the results. Below I've displayed the 3 videos recommended by our agent.

In [ ]:
from IPython.display import YouTubeVideo

In [ ]:
YouTubeVideo('nSeuq6HAJnI', width=800, height=300)


In [ ]:
YouTubeVideo('15FoQ37js_8', width=800, height=300)


In [ ]:

YouTubeVideo('TjzHL1hVo2o', width=800, height=300)

If you watch the videos you will find that all three are from reputable sources, are the latest videos (in fact the first video is currently live running live from CBS news) and are all extrememly relevant.

In [ ]:
# Let's save the 2 videos which aren't currently live!
urls = ["https://www.youtube.com/watch?v=15FoQ37js_8&t=4s", "https://www.youtube.com/watch?v=TjzHL1hVo2o&t=1s"]


We will use the YoutubeAudioLoader to fetch / download the audio files.

Then, use OpenAIWhisperParser() to transcribe them to text.

In [ ]:
agent.run("what current directory am I in?") #Yes, your agent knows Linux

In [ ]:
# Directory to save audio files
save_dir = "/content/YouTube"

In [ ]:
# set a flag to switch between local and remote parsing
# change this to True if you want to use local parsing
local = False

In [ ]:

# Transcribe the videos to text
if local:
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParserLocal())
else:
    loader = GenericLoader(YoutubeAudioLoader(urls, save_dir), OpenAIWhisperParser())
docs = loader.load()

In [ ]:
# Returns a list of Documents, which can be easily viewed or parsed
docs[0].page_content[0:500]


# Building a chat bot from your YouTube videos
## Similarly to how we build a chat bot on our TOP SECRET REPORT, now that we have downloaded, transcribed and saved the most recent and reliable YouTube videos on UAPs, we can also create a Q&A system for them.

In [ ]:
pip install faiss-cpu google-search-results #Faiss is a library for efficient similarity search and clustering of dense vectors, it's an indexing method for vectorDBs


In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [ ]:
# Combine doc
combined_docs = [doc.page_content for doc in docs]
text = " ".join(combined_docs)

In [ ]:
# Split them
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
splits = text_splitter.split_text(text)

In [ ]:
# Build an index
embeddings = OpenAIEmbeddings()
vectordb = FAISS.from_texts(splits, embeddings)

In [ ]:
# Build a QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    chain_type="stuff",
    retriever=vectordb.as_retriever(),
)

In [ ]:
query = "Do we have any UAP sightings?"
qa_chain.run(query)

In [ ]:
query = "Is there any evidence of UAP crash retrival?"
qa_chain.run(query)

In [ ]:
query = "Have aliens, UFOs or any non-human body been recovered at crash sites for UAPS?"
qa_chain.run(query)

In [ ]:
query = "Do aliens exist?"
qa_chain.run(query)

In [ ]:
query = "Is there a UFO cover-up?"
qa_chain.run(query)

## To summarize, from these YouTube videos we know that there have been multiple UAP sightings reported by credible sources, including fighter pilots and naval aviators. And while we don't have conclusive evidence that aliens exist, but there is an ongoing investigation that the United States government has been covering up UFO retrieval and reverse engineering activities for decades. Experts also claim the existence of evidence such as recovered crafts and potentially even bodies related to UFO encounters.

In [ ]:
agent.run("What is the Drake equation?")

In [ ]:
#crunch the numbers
agent.run("Take the number habitable planets are in our galaxy and multiply it by 7 billion")

In [ ]:
# for a better estimation of life, run the following:
# YouTubeVideo('WX6qQM4pVFo', width=800, height=300)
# agent.run("What is the Drake Equation?")

In [ ]:
agent.run("Has the US Government ever covered anything up?")

This notebook has successfully been used to achieve 3 key goals:

1.   Agents, chains, and tools are all individually useful, but they provide the most relevant information when leveraged together.
2.   When you give your agent and LLM chains access to the internet, the quality of the retrieved query is greatly improved. More work is needed to quanitatively evaluate agents performance, but you can start by reading the intermediate steps.
3.   Between the evidence of UAPs, the probability of life in the Galaxy and the fact that the Government may not be telling us everything there is only one possible answer to the question of "Do aliens exist?..." 👀






